In [ ]:
!pip install pymorphy2
from itertools import groupby
import re
import nltk
import pandas as pd
import random
from nltk import sent_tokenize
nltk.download('punkt')


import pymorphy2
morph = pymorphy2.MorphAnalyzer()

# Функция для поиска ближайшей границы предложения
def sentence_borders (text, word_beginning):
    """Finds the closest sentence border in the text"""
  tokenized_text = sent_tokenize(text, language='russian')
  for sentence in tokenized_text:
    sentence_beginning = text.find(sentence)
    sent_list = list(sentence)
    sentence_indexes = [i + sentence_beginning - 1 for i in range(len(sent_list))]
    if int(word_beginning) in sentence_indexes:
      answer = sentence
  return answer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
def get_key(my_dict, val):
    for key, value in my_dict.items():
         if val == value:
             return key

In [ ]:
import json
import re
import string
import os
from google.colab import drive
drive.mount('/content/drive')
from os import listdir
from os.path import isfile, join
annotations, texts = [], []
WORK_DIR = "/content/drive/My Drive/Материалы по корпусу/DEMO_markup/sentiment_dataset"
for f in os.scandir(WORK_DIR):
  s = os.path.join(WORK_DIR, f)
  n = os.path.basename(f)
  name = n.split('.')[0] #извлекаем имя файла
  extension = n.split('.')[1] #извлекаем расширение файла
  samples = open(s, encoding="utf-8")
  if extension == 'ann':
    lines = []
    for line in samples:
      lines.append(line.strip('\n').strip('\ufeff').split('\t'))
    annotations.append({name: lines}) #группируем отдельно разметку и тексты
  elif extension == 'txt':
    lines = ''
    for line in samples:
      lines+=' ' + line.strip('\n')
    texts.append({name: lines})


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from copy import deepcopy
import numpy as np


def split_row(data, column):
    '''
    разбить на строки

         : param data: необработанные данные
         : param column: разделить имя столбца
    :type data: pandas.core.frame.DataFrame
    :type column: str
    '''
    row_len = list(map(len, data[column].values))
    rows = []
    for i in data.columns:
        if i == column:
            row = np.concatenate(data[i].values)
        else:
            row = np.repeat(data[i].values, row_len)
        rows.append(row)
    print(rows[0])
    return pd.DataFrame(rows)



In [ ]:
from copy import deepcopy


def split_col(data, column):
    '''
    разбить на столбцы

         : param data: необработанные данные
         : param column: разделить имя столбца
    :type data: pandas.core.frame.DataFrame
    :type column: str
    '''
    data = deepcopy(data)
    max_len = max(list(map(len, data[column].values)))  # Максимальная длина
    new_col = data[column].apply(lambda x: x + [None]*(max_len - len(x)))  # Заполните пустое значение, None не может быть заменен на np.nan
    new_col = np.array(new_col.tolist()).T  # Транспонировать
    for i, j in enumerate(new_col):
        data[column + str(i)] = j
    return data

In [ ]:
annotated_texts = [] #список из пар "разметка - текст"
for text in texts:
  for ann in annotations:
    if text.keys()==ann.keys():
      annotated_texts.append([ann, text])

In [ ]:
total_data, fixed_data = {}, {}
entity_types = ['COUNTRY', 'PERSON', 'ORGANIZATION']
for text in annotated_texts:
  sentence_based_mark_up = {}
  entities, marked_up_entities, marked_up_relations = [], [], []
  mark_up = list(text[0].values())
  full_text = list(text[1].values())
  for part in mark_up:
    for elem in part:
      if 'T' in elem[0] and elem[1].split(' ')[0] in entity_types and ';' not in str(elem):
        code = elem[0]
        beginning = int(elem[1].split(' ')[1])
        ending = int(elem[1].split(' ')[2])
        named_entity = full_text[0][beginning:ending+1]
        entities.append([elem[0], named_entity])
        for i, one in enumerate(part):

          if one[0]!=code and one[1].split(' ')[0] in ['AUTHOR_POS', 'AUTHOR_NEG'] and int(one[1].split(' ')[1]) == beginning and int(one[1].split(' ')[2]) == ending:
            in_sentence = sentence_borders(full_text[0], beginning)
            if in_sentence not in sentence_based_mark_up.keys():
              sentence_based_mark_up[in_sentence]=[[full_text[0][beginning:ending+1], one[1].split(' ')[0].split('_')[-1],  one[1].split(' ')[0], 'автор']]
            else:
              sentence_based_mark_up[in_sentence]+=([[full_text[0][beginning:ending+1], one[1].split(' ')[0].split('_')[-1],  one[1].split(' ')[0], 'автор']])

          elif 'R' in one[0] and one[1].split(' ')[0] in ['POSITIVE_TO', 'NEGATIVE_TO'] and one[1].split(':')[-1] == code:
            in_sentence = sentence_borders(full_text[0], beginning)
            for x in part:
                if one[1].split(' ')[1].split(':')[-1] == x[0] and sentence_borders(full_text[0], int(x[1].split(' ')[2])) == in_sentence:
                  for y in part:
                    if one[1].split(' ')[1].split(':')[1] == y[0] and sentence_borders(full_text[0], int(y[1].split(' ')[2])) == in_sentence:
                      if in_sentence not in sentence_based_mark_up.keys():
                        sentence_based_mark_up[in_sentence]=[[full_text[0][beginning:ending+1], one[1].split(' ')[0][:3], one[1].split(' ')[0], y[2]]]
                      else:
                        sentence_based_mark_up[in_sentence]+=[[full_text[0][beginning:ending+1], one[1].split(' ')[0][:3], one[1].split(' ')[0], y[2]]]



          # структура OPINION_RELATES_TO: Arg1:Arg2, где Arg1 = OPINION_WORD_POS/NEG или ARGUMENT_POS_NEG, Arg2 - объект
          if 'R' in one[0] and one[1].split(' ')[0] == 'OPINION_RELATES_TO' and one[1].split(':')[-1] == code and ';' not in str(one):
            in_sentence = sentence_borders(full_text[0], beginning)
            for x in part:
                if ';' not in str(x) and one[1].split(' ')[1].split(':')[-1] == x[0] and sentence_borders(full_text[0], int(x[1].split(' ')[2])) == in_sentence:
                  for y in part:
                    if one[1].split(' ')[1].split(':')[1] == y[0] and sentence_borders(full_text[0], int(y[1].split(' ')[2])) == in_sentence and y[1].split(' ')[0] in ['ARGUMENT_POS', 'ARGUMENT_NEG', 'OPINION_WORD_POS', 'OPINION_WORD_NEG']:
                      for z in part:
                        if z[1].split(' ')[0] == 'OPINION_BELONGS_TO' and z[1].split(' ')[1].split(':')[1] == y[0]:

                          for v in part:
                            if v[0] == z[1].split(' ')[2].split(':')[1]:
                              if in_sentence not in sentence_based_mark_up.keys():
                                 sentence_based_mark_up[in_sentence]=[[full_text[0][beginning:ending+1], x[1].split(' ')[0].split('_')[-1], one[1].split(' ')[0], v[2]]]
                              else:
                                 sentence_based_mark_up[in_sentence]+=[[full_text[0][beginning:ending+1], x[1].split(' ')[0].split('_')[-1], one[1].split(' ')[0], v[2]]]

                    #elif one[1].split(' ')[1].split(':')[1] == y[0] and sentence_borders(full_text[0], int(y[1].split(' ')[2])) == in_sentence and y[1].split(' ')[0] in ['ARGUMENT_POS', 'ARGUMENT_NEG']:
                      #print(y)

          else:
            in_sentence = sentence_borders(full_text[0], beginning)
            entities_in_sentence = ['автор']
            for ent in entities:
              if ent[1] in in_sentence and ent[1]!=full_text[0][beginning:ending+1]:
                entities_in_sentence.append(ent[1])
            final_entities = list(set(entities_in_sentence))
            if in_sentence not in sentence_based_mark_up.keys():
                sentence_based_mark_up[in_sentence] = [[full_text[0][beginning:ending+1], 'NEUT', 'NaN', random.choice(final_entities)]]
            else:
              if [full_text[0][beginning:ending+1], 'NEUT', 'NaN'] not in sentence_based_mark_up[in_sentence]:
                sentence_based_mark_up[in_sentence] += [[full_text[0][beginning:ending+1], 'NEUT', 'NaN', random.choice(final_entities)]]





  for sentence in sentence_based_mark_up:
    temp_list = []
    for el in sentence_based_mark_up[sentence]:
      token_one = " ".join(morph.parse(wrd)[0].inflect({'datv'}).word if morph.parse(wrd)[0].tag.case is not None else wrd for wrd in el[0].strip(' ').split(' '))
      token_two = " ".join(morph.parse(wrd)[0].inflect({'nomn'}).word if morph.parse(wrd)[0].tag.case is not None else wrd for wrd in el[3].strip(' ').split(' '))
      temp_list.append([token_one, el[1], el[2], token_two])
    sentence_based_mark_up[sentence] = temp_list
    total_data[sentence] = temp_list





In [ ]:
for instance in total_data:
  temp_list = []
  for i in range(len(total_data[instance])):
    for j in range(len(total_data[instance])):
      if total_data[instance][i][0] == total_data[instance][j][0] and (total_data[instance][i][1] == 'NEUT' or total_data[instance][j][1]=='NEUT'):
        if total_data[instance][i][1] == 'NEUT' and total_data[instance][j][1]!= 'NEUT':
          total_data[instance][i] = 'DELETE'
        elif total_data[instance][j][1] == 'NEUT' and total_data[instance][i][1]!= 'NEUT':
          total_data[instance][j] = 'DELETE'
  temp_list = [i for i in total_data[instance] if i!='DELETE']
  total_data[instance] = temp_list



In [ ]:
for instance in total_data:
  temp_list = []
  for i in range(len(total_data[instance])-1):
    if total_data[instance][i][0]==total_data[instance][i+1][0] and (total_data[instance][i][1] == 'NEUT' or total_data[instance][i+1][1]=='NEUT'):
      total_data[instance][i] = 'DELETE'
  temp_list = [i for i in total_data[instance] if i!='DELETE']
  total_data[instance] = temp_list



In [ ]:
d = ({'sentence': [i for i in total_data.keys()], 'entity': [i for i in total_data.values()]})
d = pd.DataFrame(d)

df = split_row(d, column='entity')
total_df = df.transpose()
new_df = pd.DataFrame({'sentence': total_df[0], 'entity': [i[0] for i in total_df[1]], 'polarity': [i[1] for i in total_df[1]], 'relationship type': [i[2] for i in total_df[1]], 'source': [i[3] for i in total_df[1]]}).drop_duplicates(keep='first')
new_df.head()
new_df.to_csv('test_df.csv', index=False, encoding='utf-8')



[' В Великобритании умерла женщина, отравленная Новичком Окружной госпиталь Солсбери Британская полиция возбудила дело об убийстве после того, как умерла женщина, отравленная тем же нервно-паралитическим веществом, что и бывший российский шпион в марте 2018 года.'
 ' В Великобритании умерла женщина, отравленная Новичком Окружной госпиталь Солсбери Британская полиция возбудила дело об убийстве после того, как умерла женщина, отравленная тем же нервно-паралитическим веществом, что и бывший российский шпион в марте 2018 года.'
 ' В Великобритании умерла женщина, отравленная Новичком Окружной госпиталь Солсбери Британская полиция возбудила дело об убийстве после того, как умерла женщина, отравленная тем же нервно-паралитическим веществом, что и бывший российский шпион в марте 2018 года.'
 ...
 'Спустя несколько дней удалось задержать 15-го солдата-срочника Владислава Челаха, который, по версии следствия, является убийцей и поджигателем.'
 'Ранее заявлялось, что Владислав сознался в преступ

In [ ]:
one = new_df.duplicated(subset=['sentence', 'entity'], keep=False)
two = new_df.duplicated(subset=['sentence', 'entity', 'polarity'], keep=False)
df = pd.concat([one, two], axis = 1)
indexes_for_deletion = df.index[(df[0] == True) & (df[1] == False)].tolist()
new_df['conflicts'] = ['YES' if ind in indexes_for_deletion else 'NO' for ind in new_df.index]
new_df[:20]
new_df.to_csv('df_for_generating_questions.csv', index=False, encoding='utf-8')

In [ ]:
for text in annotated_texts[:5]:
  mark_up = list(text[0].values())
  full_text = list(text[1].values())
  for part in mark_up:
    for elem in part:
      print(elem)

['T1', 'COUNTRY 2 16', 'Великобритании']
['T3', 'DATE 242 259', 'в марте 2018 года']
['T4', 'AGE 262 271', '44-летняя']
['T5', 'PERSON 272 285', 'Дон Стерджесс']
['T6', 'CITY 310 318', 'Солсбери']
['T7', 'LOCATION 322 332', 'юге Англии']
['T8', 'DATE 343 374', 'в воскресенье, 8 июля 2018 года']
['T9', 'DATE 427 434', '30 июня']
['T10', 'PERSON 465 479', 'Чарльзом Роули']
['T11', 'PROFESSION 525 540', 'Премьер-министр']
['T12', 'PERSON 556 566', 'Тереза Мэй']
['T13', 'PERSON 634 643', 'Стерджесс']
['T14', 'PERSON 670 679', 'Стерджесс']
['T15', 'PERSON 682 687', 'Роули']
['T16', 'DATE 748 755', 'в марте']
['T18', 'PERSON 793 808', 'Сергей Скрипаль']
['T19', 'PERSON 820 824', 'Юлия']
['T20', 'COUNTRY 852 860', 'Британия']
['T21', 'PERSON 901 909', 'Скрипаля']
['T22', 'COUNTRY 913 919', 'Россию']
['T23', 'CITY 921 927', 'Москва']
['T24', 'PERSON 980 989', 'Стерджесс']
['T25', 'PERSON 992 997', 'Роули']
['T26', 'PROFESSION 1071 1093', 'Министр внутренних дел']
['T27', 'PERSON 1109 1122', 'С